In [1]:
import pdb
import csv
from nltk.corpus import stopwords, twitter_samples
import numpy as np
import pandas as pd
import nltk
import string
import random
from nltk.tokenize import TweetTokenizer
from os import getcwd
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag
from collections import defaultdict
from nltk.stem import PorterStemmer
average = np.asarray([])

In [2]:
all_positive_tweets = []
all_negative_tweets = []
all_tweets = []
with open('StrictOMD.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        all_tweets.append(row)
        if (row[0] == '4'):
            all_positive_tweets.append(row[1])
        else:
            all_negative_tweets.append(row[1])

In [3]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Amin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Amin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Amin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Amin\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw to
[nltk_data]     C:\Users\Amin\AppData\Roaming\nltk_data...
[nltk_data]   Package omw is already up-to-date!


True

# after first run, run from here to the end

## read tweets

In [837]:
all_positive_tweets = []
all_negative_tweets = []
all_tweets = np.asarray([])
with open('StrictOMD.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        all_tweets = np.append(all_tweets, row[1])
        if (row[0] == '4'):
            all_positive_tweets.append(row[1])
        else:
            all_negative_tweets.append(row[1])
# make positive and negative tweets random
random.shuffle(all_tweets)
train_x = all_tweets[100:]
test_x = all_tweets[:100]
train_y = []
test_y = []
for i in train_x:
    if i in all_positive_tweets:
        train_y = np.append(train_y, [1])
    else:
        train_y = np.append(train_y, [0])
for i in test_x:
    if i in all_positive_tweets:
        test_y = np.append(test_y, [1])
    else:
        test_y = np.append(test_y, [0])

### clean tweets

In [838]:
def process_tweet(tweet):
    """Process tweet function.
    Input:
        tweet: a string containing a tweet
    Output:
        tweets_clean: a list of words containing the processed tweet
    """
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and  # remove stopwords
                word not in string.punctuation):  # remove punctuation
            # tweets_clean.append(word)
            stem_word = stemmer.stem(word)  # stemming word
            if(len(stem_word)>1):
                tweets_clean.append(stem_word)

    return tweets_clean

In [839]:

custom_tweet = "RT @Twitter @chapagain Hello There! Have a great day. :) #good #morning http://chapagain.com.np"

# print cleaned tweet
print(process_tweet(custom_tweet))

['hello', 'great', 'day', ':)', 'good', 'morn']


In [840]:
def count_tweets(result, tweets, ys):

    for y, tweet in zip(ys, tweets):
        for word in process_tweet(tweet):
            pair = (word,y)

            if pair in result:
                result[pair] += 1

            else:
                result[pair] = 1

    return result

In [841]:
freqs = count_tweets({}, train_x, train_y)

## count unique words

In [842]:
vocab = set([pair[0] for pair in freqs.keys()])# some words are in both negative an positive sentences
V = len(freqs)

## number of all positive word and negative word with repetition

In [843]:
# calculate N_pos, N_neg, V_pos, V_neg
N_pos = N_neg = V_pos = V_neg = 0
for pair in freqs.keys():
    # if the label is positive (greater than zero)
    if pair[1] > 0:
        # increment the count of unique positive words by 1
        V_pos += 1

        # Increment the number of positive words by the count for this (word, label) pair
        N_pos += freqs[pair]

    # else, the label is negative
    else:
        # increment the count of unique negative words by 1
        V_neg += 1

        # increment the number of negative words by the count for this (word,label) pair
        N_neg += freqs[pair]

## compute log prior. number of positive sentence on number of negative one

In [844]:
#Calculate D, the number of documents
D = len(train_y)

# Calculate D_pos, the number of positive documents
D_pos = (len(list(filter(lambda x: x > 0, train_y))))

# Calculate D_neg, the number of negative documents
D_neg = (len(list(filter(lambda x: x <= 0, train_y))))

# Calculate logprior
logprior = np.log(D_pos) - np.log(D_neg)

## compute logliklihood

In [845]:
def lookup(freqs, word, label):
    n = 0
    pair = (word, label)
    if (pair in freqs):
        n = freqs[pair]

    return n

In [846]:
loglikelihood = {}
for word in vocab:
    # get the positive and negative frequency of the word
    freq_pos = lookup(freqs,word,1)
    freq_neg = lookup(freqs,word,0)

    # calculate the probability that each word is positive, and negative
    p_w_pos = (freq_pos + 1) / (N_pos + V)
    p_w_neg = (freq_neg + 1) / (N_neg + V)

    # calculate the log likelihood of the word
    loglikelihood[word] = np.log(p_w_pos/p_w_neg)

In [847]:
# loglikelihood

## prediction

In [848]:
def naive_bayes_predict(tweet, logprior, loglikelihood):
    word_l = process_tweet(tweet)
    p = 0
    p += logprior
    for word in word_l:
        # check if the word exists in the loglikelihood dictionary
        if word in loglikelihood:
            # add the log likelihood of that word to the probability
            p += loglikelihood[word]

    return p

In [849]:
def test_naive_bayes(test_x, test_y, logprior, loglikelihood):

    accuracy = 0  # return this properly
    y_hats = []
    for tweet in test_x:
        if naive_bayes_predict(tweet, logprior, loglikelihood) > 0:
            y_hat_i = 1
        else:
            y_hat_i = 0

        y_hats.append(y_hat_i)

    # error is the average of the absolute values of the differences between y_hats and test_y
    error = np.mean(np.absolute(y_hats-test_y))

    # Accuracy is 1 minus the error
    accuracy = 1-error


    return accuracy


In [850]:

print("Naive Bayes accuracy =",
      (test_naive_bayes(test_x, test_y, logprior, loglikelihood)))

Naive Bayes accuracy = 0.83


In [851]:
average = np.append(average, test_naive_bayes(test_x, test_y, logprior, loglikelihood))

In [852]:
average

array([0.83, 0.84, 0.75, 0.76, 0.68, 0.8 , 0.82, 0.81, 0.82, 0.75, 0.84,
       0.83, 0.84, 0.79, 0.8 , 0.79, 0.72, 0.81, 0.74, 0.77, 0.79, 0.8 ,
       0.78, 0.74, 0.8 , 0.75, 0.86, 0.84, 0.77, 0.83, 0.86, 0.82, 0.76,
       0.79, 0.77, 0.72, 0.76, 0.82, 0.85, 0.85, 0.83, 0.78, 0.79, 0.8 ,
       0.83, 0.81, 0.8 , 0.76, 0.79, 0.83])

In [853]:
print("average for", len(average), "run :", np.mean(average))

average for 50 run : 0.7953999999999999
